pipelines chains together steps so that output of each step is used to the next step

pipelines makes it easy to apply the same preprocessing to train and test!

# without pipeline

In [55]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [56]:
df = pd.read_csv('train.csv')

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [58]:
df['Cabin'].isnull().sum()

687

In [59]:
df.shape

(891, 12)

In [60]:
df.drop(columns = ['PassengerId','Name','Ticket','Cabin'], inplace = True)

In [61]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [62]:
# train-test split
X_train , X_test , y_train , y_test = train_test_split(
                             df.drop(columns = ['Survived']),
                             df['Survived'], test_size = 0.2,
                             random_state = 42)


In [63]:
y_train

331    0
733    0
382    0
704    0
813    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 712, dtype: int64

In [64]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [65]:
# applying imputation
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [78]:
# one hot encoding on sex and Embarked
# (embarked has null values)
ohe_sex = OneHotEncoder(sparse_output = False ,handle_unknown = 'ignore')
ohe_embarked = OneHotEncoder(sparse_output = False ,handle_unknown = 'ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

In [79]:
X_train_rem = X_train.drop(columns = ['Sex','Age','Embarked'])
X_test_rem = X_test.drop(columns = ['Sex','Age','Embarked'])


In [80]:
X_train_transformed = np.concatenate((X_train_rem, X_train_age , X_train_sex , X_train_embarked), axis = 1)
X_test_transformed = np.concatenate((X_test_rem, X_test_age , X_test_sex , X_test_embarked), axis = 1)


In [81]:
X_test_transformed.shape

(179, 13)

In [82]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed , y_train)

DecisionTreeClassifier()

In [83]:
y_pred = clf.predict(X_test_transformed)

In [84]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7653631284916201

In [85]:
import pickle

In [89]:
pickle.dump(ohe_sex,open('ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('ohe_embarked.pkl','wb'))
pickle.dump(clf,open('clf.pkl','wb'))
